In [1]:
''' Dependencies '''
from sklearn.decomposition import PCA
from tqdm.auto import tqdm
import warnings
import pandas as pd
import numpy as np
import scipy
import sys
import statsmodels.api as sm
from tqdm import tqdm as pbar

# my scripts
from pyaldata import * 
import preprocess
from utils import *

# Plotting
from IPython.display import display, set_matplotlib_formats, HTML
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.colors
import matplotlib.pyplot as plt
import seaborn as sns
import random

db = '#283db5' # Darkblue default
ddb = '#0c112e'
g = '#767676'
cs = ['#43D789', '#801607', '#8DB8ED', '#94B0B6', '#e42c12', '#005CA8', '#127340', '#111851'] # Line colors
cmap = matplotlib.colors.ListedColormap(['#e4e4e4', 'b', 'g']) # Color for False and True
#cmap = 'Blues'

sns.set(rc={'figure.figsize':(10, 5)})
sns.set_style('ticks', rc={ 'figure.facecolor': 'none', 'axes.facecolor':'none'})
sns.set_context('notebook', rc={'lines.linewidth':1.5})

%config InlineBackend.figure_format = 'retina'

/Users/Daphne/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
main_dict = np.load('/Users/Daphne/Data/main_dict.npy', allow_pickle='TRUE').item()

PCs_surr = np.load('surr_across_neurons.npy') # Load PC surrogates

In [3]:
PCs_surr = PCs_surr[:50, :, :] # take a subset

In [4]:
''' Parameters '''
OTHER_ARRAY_D = 50 # Distance value to indicate other array

TYPE_ANALYSIS = 'pooled' # alternative: 'm1', 'pmd' #TODO

SESSIONS = ['Chewie_CO_VR_2016-09-09', 'Chewie_CO_VR_2016-09-12', 'Chewie_CO_VR_2016-09-14', 'Chewie_CO_VR_2016-10-06', 
            'Chewie_CO_FF_2016-09-15', 'Chewie_CO_FF_2016-09-21', 'Chewie_CO_FF_2016-10-05', 'Chewie_CO_FF_2016-10-07',
            'Mihili_CO_VR_2014-03-04', 'Mihili_CO_VR_2014-03-06', 'Mihili_CO_FF_2014-02-03', 'Mihili_CO_FF_2014-02-17', 
            'Mihili_CO_FF_2014-02-18', 'Mihili_CO_FF_2014-03-07', 'Mihili_CO_VR_2014-03-03']

THRESHOLDS_VARE = np.round(np.arange(0.2, 0.95, 0.025), 3)
INTERVALS = [(0, 2), (0, 4)] # Fixed within intervals

''' Load unit guides '''
M1_UG = np.load('/Users/Daphne/Data/M1_UG.npy')
PMD_UG = np.load('/Users/Daphne/Data/PMD_UG.npy')

''' Experimental sessions and electrode maps '''
m1_emap  = localize_elecs(read_cmp(file_path='/Users/Daphne/Data/Chewie Left M1 SN 6250-001474.cmp'),  elecs=range(1,97))
pmd_emap = localize_elecs(read_cmp(file_path='/Users/Daphne/Data/Chewie Left PMd SN 6251-001469.cmp'), elecs=range(1,97))

## Start

In [5]:
''' Initialize surrogate dictionary '''
surr_dict = {}
for s in range(PCs_surr.shape[0]): surr_dict[f'{s}'] = {}

In [6]:
''' Add PCs to each surrogate '''
for i, s in enumerate(surr_dict.keys()):
    surr_dict[s]['pcs'] = PCs_surr[i, :, :]

In [7]:
''' Prepare dictionaries to store each step for level of var exp '''
for s in surr_dict.keys():
    surr_dict[s]['vare'] = {}
    for th in THRESHOLDS_VARE:
        surr_dict[s]['vare'][f'{th}'] = { 'distances_df': {} }

In [8]:
# Define components range from original data (session = 2016-09-09)
components_range = [np.argmax(main_dict['2016-09-09']['model'].explained_variance_ratio_.cumsum() > THRESHOLDS_VARE[i]) for i in range(len(THRESHOLDS_VARE))]

In [9]:
for s in pbar(surr_dict.keys()):
    
    for i, r in enumerate(components_range):

        if r == 0: r += 2 # Can't compute correlation between two values
        elif r == 1: r += 1 
            
        L_surr = surr_dict[s]['pcs'][:, :r] # Get the first r PCs

        # Get correlations and physical distances
        C, PD, A, _ = compute_stat_and_phys_distances(L_surr, M1_UG, PMD_UG, m1_emap, pmd_emap)
        
        # Convert to dataframe
        df = pd.DataFrame(data={'correlation': C, 'distance': PD, 'on array': A})
        df['category'] = df['distance'].apply(lambda d: 'same electrode' if d == 0 else ('same array' if d < OTHER_ARRAY_D else ('other array')))
        
        # Store dataframe in surrogate dictionary
        surr_dict[s]['vare'][f'{THRESHOLDS_VARE[i]}']['distances_df'] = df

100%|██████████| 50/50 [30:28<00:00, 36.57s/it]


In [10]:
''' Perform LS regressions on each surrogate '''

for s in pbar(surr_dict.keys()):
    
    for v in surr_dict[s]['vare'].keys(): 
        
        # Empirical and surrogate data
        variants = {
          'Surr' : surr_dict[s]['vare'][f'{v}']['distances_df'],
        }
        # Dummy functions
        int_dum = lambda df, lb, ub: df['distance'].apply(lambda x: 1 if lb < x <= ub else 0)
        on_arr = lambda df, arr: df['on array'].apply(lambda x: 1 if x == arr else 0)
        same_ele = lambda df: df['distance'].apply(lambda x: 1 if x == 0 else 0)
        same_arr = lambda df: df['distance'].apply(lambda x: 1 if 0 < x < OTHER_ARRAY_D else 0)

        # Analyses methods
        analyses = {
          'single_cat' : {
            'fixed cols' : {
              'OA Constant' : lambda df: [1] * len(df),
              'SE (d = 0)' : same_ele,
              'SA (d exists)' : same_arr,
            },
            'variable cols' : {}
          }}
        
        for study, df in variants.items():
            
            for analysis, cols_dict in analyses.items():
                df_X = pd.DataFrame()
                for col_name, col_func in cols_dict['fixed cols'].items():
                    df_X[col_name] = col_func(df)
                for (lb, ub) in INTERVALS:
                    for col_name, col_func in cols_dict['variable cols'].items():
                        df_X[col_name + f'({lb:.1f}, {ub:.1f}]'] = col_func(df, lb, ub)
                df_X = df_X.reindex(sorted(df_X.columns), axis=1)
                res = sm.OLS(df.correlation, df_X, hasconst=True).fit()
                title = f'{study}_{analysis}'
                
                surr_dict[s]['vare'][f'{v}'][title] = res

100%|██████████| 50/50 [00:47<00:00,  1.05it/s]


### Extract betas for all three groups and save


In [14]:
# Extract betas and pvalues for single cat 

beta0_surr = []
for s in surr_dict.keys():
    beta0_surr.append([surr_dict[s]['vare'][v]['Surr_single_cat'].params[0] for v in surr_dict[s]['vare'].keys()])
beta0_surr = np.array(beta0_surr)

beta1_surr = []
for s in surr_dict.keys():
    beta1_surr.append([surr_dict[s]['vare'][v]['Surr_single_cat'].params[1] for v in surr_dict[s]['vare'].keys()])
beta1_surr = np.array(beta1_surr)

beta2_surr = []
for s in surr_dict.keys():
    beta2_surr.append([surr_dict[s]['vare'][v]['Surr_single_cat'].params[2] for v in surr_dict[s]['vare'].keys()])
beta2_surr = np.array(beta2_surr)

In [19]:
np.array(beta0_surr).shape

(50, 30)

In [12]:
# Save them
beta0_surr = np.save('beta0_surr.npy', beta0_surr)
beta1_surr = np.save('beta1_surr.npy', beta1_surr)
beta2_surr = np.save('beta2_surr.npy', beta2_surr)

### Extract p, t values and save

In [ ]:
pvals_surr = []
tvals_surr = []

for s in surr_dict.keys():
    
    # SURROGATES
    pvals_surr.append([surr_dict[s]['vare'][v]['Surr_single_cat'].pvalues.values for v in surr_dict[s]['vare'].keys()])
    tvals_surr.append([surr_dict[s]['vare'][v]['Surr_single_cat'].tvalues.values for v in surr_dict[s]['vare'].keys()])
    
pvals_surr = np.array(pvals_surr) # (should be 50 x 30)
tvals_surr = np.array(tvals_surr)

In [ ]:
# Save them
np.save('pvals_surr', pvals_surr)
np.save('tvals_surr', tvals_surr)

### Perform rank sum test on surrogates

In [20]:
sessions_twoSided_SE_SR = []
sessions_twoSided_SE_OR = []
sessions_twoSided_SR_OR = []

sessions_lt_SE_SR = []
sessions_lt_SE_OR = []
sessions_lt_SR_OR = []

sessions_gt_SE_SR = []
sessions_gt_SE_OR = []
sessions_gt_SR_OR = []

for s in surr_dict.keys(): # for each session
        
    for v in surr_dict[s]['vare'].keys(): # for each threshold of variance explained
        
        df = surr_dict[s]['vare'][v]['distances_df'] # get df for that level of exp var
        
        # Get 3 distributions
        corrs_SE = df.loc[df['category'] == 'same electrode']['correlation'].values
        corrs_SR = df.loc[df['category'] == 'same array']['correlation'].values
        corrs_OR = df.loc[df['category'] == 'other array']['correlation'].values
        
        # get lists of tuples (30 x 15) = (test statistic, pval)
        sessions_twoSided_SE_SR.append(scipy.stats.ranksums(corrs_SE, corrs_SR, alternative='two-sided') * 1)
        sessions_twoSided_SE_OR.append(scipy.stats.ranksums(corrs_SE, corrs_OR, alternative='two-sided') * 1)
        sessions_twoSided_SR_OR.append(scipy.stats.ranksums(corrs_SR, corrs_OR, alternative='two-sided') * 1)
        
        # LESS THAN
        sessions_lt_SE_SR.append(scipy.stats.ranksums(corrs_SE, corrs_SR, alternative='less') * 1)
        sessions_lt_SE_OR.append(scipy.stats.ranksums(corrs_SE, corrs_OR, alternative='less') * 1)
        sessions_lt_SR_OR.append(scipy.stats.ranksums(corrs_SR, corrs_OR, alternative='less') * 1)
        
        # LESS THAN
        sessions_gt_SE_SR.append(scipy.stats.ranksums(corrs_SE, corrs_SR, alternative='greater') * 1)
        sessions_gt_SE_OR.append(scipy.stats.ranksums(corrs_SE, corrs_OR, alternative='greater') * 1)
        sessions_gt_SR_OR.append(scipy.stats.ranksums(corrs_SR, corrs_OR, alternative='greater') * 1)

In [25]:
np.save('surr_gt_SE_SR', sessions_gt_SE_SR)

In [26]:
np.save('surr_gt_SE_OR', sessions_gt_SE_OR)

In [24]:
np.save('surr_gt_SR_OR', sessions_gt_SR_OR)

In [ ]:
# #Generating surrogates with one session
# W = main_dict['2016-09-09']['pcs']
# N_surrogates = 100
# surr_data = []

# for i in range(N_surrogates):
    
#     surr_data.append(np.random.permutation(W))
#np.save('surr_across_neurons', Surr_across_neurons)

In [ ]:
# #rand_row_idx = random.sample(range(X_concat.shape[0]), X_concat.shape[0])
# rand_row_idx = random.sample(range(A.shape[0]), A.shape[0])

# print(rand_row_idx)

# # Randomize the rows of the data matrix
# X_rand_neurons = A[rand_row_idx, :]